In [70]:
import pandas as pd
import requests_html
from bs4 import BeautifulSoup
import datetime
import time

In [71]:
df = pd.read_excel("비브릭 운용 메타마스크 지갑 Ethereum(Net) transfer history 2022년.xls")
df.head(5)

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,ContractAddress,Value_IN(ETH),Value_OUT(ETH),CurrentValue @ $2037.08/Eth,TxnFee(ETH),TxnFee(USD),Historical $Price/Eth,Status,ErrCode,Method
0,0x7b0698e803d17d1c97124d6ffa54b68c332ab64e020f...,13918264,1641023315,2022-01-01 07:48:35,0xe3031c1bfaa7825813c562cbdcc69d96fcad2087,0x8f27fa136ff4da1796368380019029d81d12d06d,NaN,6563.112738,0.000000,1.336959e+07,0.001392,2.836035,3766.74,NaN,NaN,Transfer
1,0xa8690bc8ddd8416e643b3867c000fdd3db89c23071e8...,13918287,1641023648,2022-01-01 07:54:08,0x8f27fa136ff4da1796368380019029d81d12d06d,0xa0c68c638235ee32657e8f720a23cec1bfc77c77,NaN,0.000000,6562.186781,1.336770e+07,0.004729,9.632746,3766.74,NaN,NaN,Deposit Ether For
2,0x3eb0f62031565de709770d2e916bffe8eb9050d17a3d...,13920025,1641046386,2022-01-01 14:13:06,0x8f27fa136ff4da1796368380019029d81d12d06d,0x1111111254fb6c44bac0bed2854e76f90643097d,NaN,0.000000,0.000000,0.000000e+00,0.061185,124.638716,3766.74,NaN,NaN,0x7c025200
3,0x99e49b872d6accbf9274b5a05ffbf1f076398db08263...,13920054,1641046681,2022-01-01 14:18:01,0x8f27fa136ff4da1796368380019029d81d12d06d,0xa47c8bf37f92abed4a126bda807a7b7498661acd,NaN,0.000000,0.000000,0.000000e+00,0.001887,3.844066,3766.74,NaN,NaN,Burn
4,0x2b6872407b845147d1640ae4cdbc289b45df72989bc2...,13923631,1641094380,2022-01-02 03:33:00,0x8f27fa136ff4da1796368380019029d81d12d06d,0x1111111254fb6c44bac0bed2854e76f90643097d,NaN,0.000000,0.000000,0.000000e+00,0.093928,191.338738,3830.92,NaN,NaN,0x7c025200


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Txhash                       149 non-null    object        
 1   Blockno                      149 non-null    int64         
 2   UnixTimestamp                149 non-null    int64         
 3   DateTime                     149 non-null    datetime64[ns]
 4   From                         149 non-null    object        
 5   To                           149 non-null    object        
 6   ContractAddress              0 non-null      float64       
 7   Value_IN(ETH)                149 non-null    float64       
 8   Value_OUT(ETH)               149 non-null    float64       
 9   CurrentValue @ $2037.08/Eth  149 non-null    float64       
 10  TxnFee(ETH)                  149 non-null    float64       
 11  TxnFee(USD)                  149 non-null    

In [73]:
# Null 처리
df = df[df['Status'].isnull()==True]

# KST Time도 더해줘야지
df['KST Time (GMT+9)'] = df.apply(lambda r: r['DateTime'] + datetime.timedelta(hours=9), axis = 1)

In [74]:
# 1. 엑셀에 있으면 가져오기
# 2. 없으면 크롤링하고, 엑셀에 추가하기
# 3. 구글스프레드시트 API

In [75]:
# 한줄이면 가능
df = df.loc[:,['KST Time (GMT+9)','Txhash','Method','From','To']]
df.head()



,KST Time (GMT+9),Txhash,Method,From,To
0,2022-01-01 16:48:35,0x7b0698e803d17d1c97124d6ffa54b68c332ab64e020f...,Transfer,0xe3031c1bfaa7825813c562cbdcc69d96fcad2087,0x8f27fa136ff4da1796368380019029d81d12d06d
1,2022-01-01 16:54:08,0xa8690bc8ddd8416e643b3867c000fdd3db89c23071e8...,Deposit Ether For,0x8f27fa136ff4da1796368380019029d81d12d06d,0xa0c68c638235ee32657e8f720a23cec1bfc77c77
2,2022-01-01 23:13:06,0x3eb0f62031565de709770d2e916bffe8eb9050d17a3d...,0x7c025200,0x8f27fa136ff4da1796368380019029d81d12d06d,0x1111111254fb6c44bac0bed2854e76f90643097d
3,2022-01-01 23:18:01,0x99e49b872d6accbf9274b5a05ffbf1f076398db08263...,Burn,0x8f27fa136ff4da1796368380019029d81d12d06d,0xa47c8bf37f92abed4a126bda807a7b7498661acd
4,2022-01-02 12:33:00,0x2b6872407b845147d1640ae4cdbc289b45df72989bc2...,0x7c025200,0x8f27fa136ff4da1796368380019029d81d12d06d,0x1111111254fb6c44bac0bed2854e76f90643097d


In [76]:
# Datetime 내림차순 정렬
df.sort_values(by=["KST Time (GMT+9)"], ascending = False, inplace=True)
df.head()

,KST Time (GMT+9),Txhash,Method,From,To
148,2022-05-14 12:15:43,0x685d163fa8189f424d7b353ccdb0a57a3498a85929ce...,Transfer,0x8f27fa136ff4da1796368380019029d81d12d06d,0x8dbe9c44a42050da44f075149c1ff49b18bcaf3f
147,2022-05-02 10:48:14,0xf8c3e667085bc732ed5b566420eddee11e4a71c55446...,Transfer,0x8f27fa136ff4da1796368380019029d81d12d06d,0xd16eebdb4f8804b66ab4b2c536917154d9a47e82
146,2022-05-02 10:44:52,0x6fc24065423ecba06ba37e4f89a433dac9fe75bb0fe5...,Exchange,0x8f27fa136ff4da1796368380019029d81d12d06d,0xdc24316b9ae028f1497c275eb9192a3ea0f67022
145,2022-05-02 10:43:56,0xdb2e850b168894409ed337aaaf56da10796e455caa99...,Approve,0x8f27fa136ff4da1796368380019029d81d12d06d,0xae7ab96520de3a18e5e111b5eaab095312d7fe84
144,2022-05-02 10:41:59,0x8591a9da800eab418c08f77326aad04330ffa7aa9745...,Withdraw,0x8f27fa136ff4da1796368380019029d81d12d06d,0xa2f987a546d4cd1c607ee8141276876c26b72bdf


In [77]:
# 4. 타겟 지갑 주소를 from, to와 비교한다
main_wallet_address = "0x8f27fa136ff4da1796368380019029d81d12d06d"
# 돌아가면서 하려면 새로운 열을 만드는게 좋을듯
target_address = 0
# 두번에 돌아가서 
df['TargetAddress'] = df.apply(lambda r: r['To'] if r['From'] == main_wallet_address else r['From'], axis = 1)

# df['From']
# df['To']
df.head(5)


,KST Time (GMT+9),Txhash,Method,From,To,TargetAddress
148,2022-05-14 12:15:43,0x685d163fa8189f424d7b353ccdb0a57a3498a85929ce...,Transfer,0x8f27fa136ff4da1796368380019029d81d12d06d,0x8dbe9c44a42050da44f075149c1ff49b18bcaf3f,0x8dbe9c44a42050da44f075149c1ff49b18bcaf3f
147,2022-05-02 10:48:14,0xf8c3e667085bc732ed5b566420eddee11e4a71c55446...,Transfer,0x8f27fa136ff4da1796368380019029d81d12d06d,0xd16eebdb4f8804b66ab4b2c536917154d9a47e82,0xd16eebdb4f8804b66ab4b2c536917154d9a47e82
146,2022-05-02 10:44:52,0x6fc24065423ecba06ba37e4f89a433dac9fe75bb0fe5...,Exchange,0x8f27fa136ff4da1796368380019029d81d12d06d,0xdc24316b9ae028f1497c275eb9192a3ea0f67022,0xdc24316b9ae028f1497c275eb9192a3ea0f67022
145,2022-05-02 10:43:56,0xdb2e850b168894409ed337aaaf56da10796e455caa99...,Approve,0x8f27fa136ff4da1796368380019029d81d12d06d,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,0xae7ab96520de3a18e5e111b5eaab095312d7fe84
144,2022-05-02 10:41:59,0x8591a9da800eab418c08f77326aad04330ffa7aa9745...,Withdraw,0x8f27fa136ff4da1796368380019029d81d12d06d,0xa2f987a546d4cd1c607ee8141276876c26b72bdf,0xa2f987a546d4cd1c607ee8141276876c26b72bdf


In [78]:
# Please enable Javascript
# https://theautomatic.net/2019/01/19/scraping-data-from-javascript-webpage-python/

In [79]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome('./chromedriver.exe', options=options)
# driver = webdriver.Chrome('./chromedriver.exe')

/var/folders/zd/w2kt821j22xcwhd5hppdy4n80000gn/T/ipykernel_5646/2224891543.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [80]:
etherscan_url = "https://etherscan.io/address/"
target_address = "0x8dbe9c44a42050da44f075149c1ff49b18bcaf3f"

driver.get(etherscan_url + target_address)

In [81]:
from selenium.webdriver.common.by import By
time.sleep(0.01) # 아니면 async를 조지는것도 좋다
address_or_contract_selector = driver.find_element(By.CSS_SELECTOR, "#content > div.container.py-3 > div > div.mb-3.mb-lg-0 > h1")
address_or_contract_selector.text
# address_or_contract = address_or_contract_selector.text.split()[0]
# split으로 둘이 나눠버려!

'Address 0x8dbE9c44A42050da44f075149C1ff49B18bCaF3f\n3'